# Base

In [ ]:
skus = [
    'Lambsquarters',
    'PalmerAmaranth',
    'Waterhemp',
    'MorningGlory',
    'Purslane',
    'Goosegrass',
    'Carpetweed',
    'SpottedSpurge',
    'Ragweed',
    'Eclipta',
]

sku_dir_real_dict = {}
sku_dir_real_hint_dict = {}

for sku in skus:
    sku_dir_real_dict[sku] = fr'D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_origin_{sku}_block\target'
    sku_dir_real_hint_dict[sku] = fr'D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_origin_{sku}_block\source'
skus = [
    'Lambsquarters',
    'Waterhemp',
]
for sku in skus:
    sku_dir_real_dict[sku] = fr'D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_crop1500_{sku}\target'
    sku_dir_real_hint_dict[sku] = fr'D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_crop1500_{sku}\source'
skus = [
    'Purslane',
]
for sku in skus:
    sku_dir_real_dict[sku] = fr'D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_crop1500_{sku}_block\target'
    sku_dir_real_hint_dict[sku] = fr'D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_crop1500_{sku}_block\source'

sku_dir_fake_dict = {}
for sku in sku_dir_real_dict:
    sku_dir_fake_dict[sku] = fr'D:\Dataset\WeedData\weed_10_species\train_image_for_controlnet_all\train2017_{sku}_detection'

sku_dir_real_dict = dict(sorted(sku_dir_real_dict.items()))
sku_dir_fake_dict = dict(sorted(sku_dir_fake_dict.items()))
# sku_dir_real_dict, sku_dir_fake_dict

In [ ]:
sku_dir_real_dict = {}
sku_dir_synthetic_dict = {}
sku_dir_real_dict['train'] = r'weed_10_species\train2017_split_3_same_im_1024'
sku_dir_synthetic_dict['train'] = r'weed_10_species\train2017_split_3_generated_im_res512_script_edge_blend'

In [ ]:
sku_dir_real_dict = {}
sku_dir_synthetic_dict = {}

sku_dir_real_dict['train'] = r'weed_10_species\train2017_real_object_in_box'

sku_dir_synthetic_dict['train'] = r'generated_10_species\train2017_split_1_generated_im_res512_script_edge_blend_all_object_in_box'

In [ ]:
skus = [
    'Lambsquarters',
    'PalmerAmaranth',
    'Waterhemp',
    'MorningGlory',
    'Purslane',
    'Goosegrass',
    'Carpetweed',
    'SpottedSpurge',
    'Ragweed',
    'Eclipta',
]
sku_dir_real_dict = {}
sku_dir_synthetic_dict = {}
for sku in skus:
    sku_dir_real_dict[sku] = fr'weed_10_species\train2017_real_object_in_box\{sku}'
for sku in sku_dir_real_dict:
    sku_dir_synthetic_dict[sku] = fr'generated_10_species\train2017_split_1_generated_im_res512_script_edge_blend_all_mask_ann_only_bioclip_object_in_box\{sku}'
sku_dir_real_hint_dict = sku_dir_real_dict

# PSNR

In [1]:
import math
import numpy as np

def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


# FID

In [ ]:
import torch
import os
_ = torch.manual_seed(123)
from torchmetrics.image.fid import FrechetInceptionDistance
fid = FrechetInceptionDistance(feature=64)
imgs_dist1 = torch.randint(0, 200, (10, 3, 299, 299), dtype=torch.uint8)
imgs_dist2 = torch.randint(100, 255, (10, 3, 299, 299), dtype=torch.uint8)
fid.update(imgs_dist1, real=True)
fid.update(imgs_dist2, real=False)
fid.compute()

tensor(12.7626)

In [3]:
def list_dir(path, list_name, extension, return_names=False):
    import os
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            list_dir(file_path, list_name, extension)
        else:
            if file_path.endswith(extension):
                if return_names:
                    list_name.append(file)
                else:
                    list_name.append(file_path)
    try:
        list_name = sorted(list_name, key=lambda k: int(os.path.split(k)[1].split(extension)[0].split('_')[-1]))
    except Exception as e:
        print(e)
    return list_name

In [ ]:
"""
FID for each sku / Overall FID
"""
import torch
from tqdm import tqdm
import cv2
import random
from torchmetrics.image.fid import FrechetInceptionDistance
import numpy as np
import os

def preprocess_image1(image):
    image = torch.tensor(image).unsqueeze(0)
    # image = image.permute(0, 3, 1, 2) / 255.0
    image = image.permute(0, 3, 1, 2)
    return image
compute_each_sku = False
fid = FrechetInceptionDistance(feature=64, normalize=True)
sku_FID_dict = {}
for sku in sku_dir_real_dict:
    print(sku)
    src_dir = sku_dir_real_dict[sku]
    src_hint_dir = sku_dir_real_hint_dict[sku]

    dst_dir = sku_dir_synthetic_dict[sku]
    print(src_dir)
    print(dst_dir)
    # fid = FrechetInceptionDistance(feature=64, normalize=False)
    if compute_each_sku:
        fid = FrechetInceptionDistance(feature=64, normalize=True)

    update_step = 100
    src_im_paths = []
    src_ann_paths = list_dir(src_hint_dir, [], '.jpg')
    for i_im, src_ann_path in tqdm(enumerate(src_ann_paths)):
            im_name_base = src_ann_path.split('.')[0]
            _, im_name = os.path.split(src_ann_path)
            im_name = im_name_base+'.jpg'
            src_im_path = os.path.join(src_dir, im_name)
            src_im_paths.append(src_im_path)

    dst_im_paths = []
    dst_ann_paths = list_dir(dst_dir, [], '.jpg')
    for i_im, dst_ann_path in tqdm(enumerate(dst_ann_paths)):
            im_name_base = dst_ann_path.split('.')[0]
            im_dir, im_name = os.path.split(dst_ann_path)
            im_name = im_name_base+'.jpg'
            dst_im_path = os.path.join(im_dir, im_name)
            dst_im_paths.append(dst_im_path)

    random.seed(1)
    random.shuffle(src_im_paths)
    random.shuffle(dst_im_paths)
    im_num = -1
    # im_num = 1000
    # im_num = min(len(src_im_paths), len(dst_im_paths))
    im_size = 512
    original_images = []
    for i_im, src_im_path in tqdm(enumerate(src_im_paths)):
        if im_num > 0 and i_im>=im_num:
            if len(original_images) > 0:
                original_images=torch.cat([preprocess_image1(image) for image in original_images])
                fid.update(original_images, real=True)
                original_images = []
            break
        original_image=cv2.imread(src_im_path)
        original_image = cv2.resize(original_image, (im_size,im_size))
        original_images.append(original_image)
        if i_im%update_step==0 or i_im == len(src_im_paths)-1:
            original_images=torch.cat([preprocess_image1(image) for image in original_images])
            fid.update(original_images, real=True)
            original_images = []

    edited_images =[]
    for i_im, dst_im_path in tqdm(enumerate(dst_im_paths)):
        if im_num > 0 and i_im>=im_num:
            if len(edited_images) > 0:
                edited_images=torch.cat([preprocess_image1(image) for image in edited_images])
                fid.update(edited_images, real=False)
                edited_images =[]
            break
        edited_image=cv2.imread(dst_im_path)
        edited_image = cv2.resize(edited_image, (im_size,im_size))
        edited_images.append(edited_image)
        if i_im%update_step==0 or i_im == len(dst_im_paths)-1:
            edited_images=torch.cat([preprocess_image1(image) for image in edited_images])
            fid.update(edited_images, real=False)
            edited_images =[]
    if compute_each_sku:
        FID = float(fid.compute())
        print(f"FID: {round(FID, 2)}")
        sku_FID_dict[sku] = round(FID, 2)
    # original_images=torch.cat([preprocess_image1(image) for image in original_images])
    # edited_images=torch.cat([preprocess_image1(image) for image in edited_images])
    # # All images will be resized to 299 x 299 in official doc
    # fid.update(original_images, real=True)
    # fid.update(edited_images, real=False)
if not compute_each_sku:
    FID = float(fid.compute())
    print(f"FID: {round(FID, 2)}")
print(sku_FID_dict)

# IS

In [5]:
import torch
from tqdm import tqdm
import cv2
import random
from torchmetrics.image.inception import InceptionScore
import numpy as np

import torch
_ = torch.manual_seed(123)
from torchmetrics.image.inception import InceptionScore
inception = InceptionScore()
# generate some images
imgs = torch.randint(0, 255, (100, 3, 299, 299), dtype=torch.uint8)
inception.update(imgs)
inception.compute()

In [ ]:
"""
Overall / each sku IS
"""
import torch
_ = torch.manual_seed(123)
from torchmetrics.image.inception import InceptionScore
# inception = InceptionScore()
inception = InceptionScore(normalize=True)

imgs =[]
update_step = 100
compute_each_sku = False
# for sku in sku_dir_fake_dict:
for sku in sku_dir_synthetic_dict:
    print(sku)
    dst_dir = sku_dir_synthetic_dict[sku]
    print(dst_dir)
    dst_im_paths = []
    if compute_each_sku:
        inception = InceptionScore(normalize=True)
    # dst_ann_paths = list_dir(dst_dir, [], '.xml')
    dst_ann_paths = list_dir(dst_dir, [], '.jpg')
    for i_im, dst_ann_path in tqdm(enumerate(dst_ann_paths)):
            im_name_base = dst_ann_path.split('.')[0]
            im_dir, im_name = os.path.split(dst_ann_path)
            im_name = im_name_base+'.jpg'
            # dst_im_path = os.path.join(im_dir, im_name)
            dst_im_path = dst_ann_path
            dst_im_paths.append(dst_im_path)

    print(dst_dir)
    print(len(dst_im_paths))
    # random.seed(0)
    random.seed(1)
    random.shuffle(dst_im_paths)
    # im_num = 1000
    im_num = len(dst_im_paths)
    im_size = 512
    for i_im, dst_im_path in tqdm(enumerate(dst_im_paths)):
        if i_im>=im_num:
            imgs=torch.cat([preprocess_image1(image) for image in imgs])
            inception.update(imgs)
            imgs = []
            break
        img=cv2.imread(dst_im_path)
        img = cv2.resize(img, (im_size,im_size))
        imgs.append(img)
        if i_im%update_step==0 or i_im == len(dst_im_paths)-1:
            imgs=torch.cat([preprocess_image1(image) for image in imgs])
            inception.update(imgs)
            imgs = []
    if compute_each_sku:
        print('IS:', inception.compute())
if not compute_each_sku:
    print('IS:', inception.compute())


Lambsquarters
D:\BoyangDeng\StableDiffusion\test\generated_10_species\train2017_split_1_generated_im_res512_script_edge_blend_all_mask_ann_only_bioclip_object_in_box\Lambsquarters


541it [00:00, ?it/s]


D:\BoyangDeng\StableDiffusion\test\generated_10_species\train2017_split_1_generated_im_res512_script_edge_blend_all_mask_ann_only_bioclip_object_in_box\Lambsquarters
541


541it [00:38, 13.91it/s]


PalmerAmaranth
D:\BoyangDeng\StableDiffusion\test\generated_10_species\train2017_split_1_generated_im_res512_script_edge_blend_all_mask_ann_only_bioclip_object_in_box\PalmerAmaranth


...

Eclipta
D:\BoyangDeng\StableDiffusion\test\generated_10_species\train2017_split_1_generated_im_res512_script_edge_blend_all_mask_ann_only_bioclip_object_in_box\Eclipta


3303it [00:00, 65147.99it/s]


D:\BoyangDeng\StableDiffusion\test\generated_10_species\train2017_split_1_generated_im_res512_script_edge_blend_all_mask_ann_only_bioclip_object_in_box\Eclipta
3303


3303it [03:52, 14.23it/s]

IS: (tensor(3.7450), tensor(0.1136))


# Improved Pre and Rec

In [16]:
from improved_precision_recall import IPR

In [ ]:
for sku in sku_dir_fake_dict:
    print(sku)
    print(sku_dir_real_dict[sku])
    print(sku_dir_fake_dict[sku])

In [2]:
! python improved_precision_recall.py D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_origin_PalmerAmaranth_block\target D:\Dataset\WeedData\weed_10_species\train_image_for_controlnet\train2017_PalmerAmaranth_detection

loading vgg16 for improved precision and recall...done
precision: 0.0033333333333333335
recall: 0.27852998065764023
found 1 images in D:\BoyangDeng\StableDiffusion\ControlNet\training\weedall_origin_PalmerAmaranth_block\target
realism of first image in real: 2750.8298517469375
found 1 images in D:\Dataset\WeedData\weed_10_species\train_image_for_controlnet\train2017_PalmerAmaranth_detection
realism of first image in fake: 0.6812681117764692


c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\AFSALab\.conda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

extracting features of 517 images: 100%|██████████| 11/11 [01:55<00:00, 10.46s/it]

extracting features of 1200 images: 100%|██████████| 24/24 [00:39<00:00,  1.66s/it]

computing precision...: 100%|██████████| 1200/1200 [00:00<00:00, 119982.95it/s]

computing recall...: 100%|██████████| 517/517 [00:00<00:00, 51701.28it/s]

extracting features of 1 im

In [ ]:
ipr = IPR(args.batch_size, args.k, args.num_samples)
ipr.compute_manifold_ref(args.path_real)  
metric = ipr.precision_and_recall(images)
print('precision =', metric.precision)
print('recall =', metric.recall)

# CLIP Score

In [ ]:
import torch
from torchmetrics.multimodal.clip_score import CLIPScore
metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
score = metric(torch.randint(255, (3, 224, 224), generator=torch.manual_seed(42)), "a photo of a cat")
score.detach()